In [ ]:
import openmatrix as omx
import pandas as pd
import numpy as np
import yaml
from utility import *
import warnings
warnings.filterwarnings("ignore")

#### Details
Jobs accessible from people's homes : 
    count of jobs accessible to each zone within isochrone thresholds (both actual and perceived). Weighted average by workforce in origin zone.
    PP metric: yes
    Multi or Single path : Single
    Modes : rail inclusive (Does this mean only rail OD pairs (IVTHWY + IVTCOM > 0) ?)
    Purpose split: No
    Period splits: Yes, average weekday and annual
    Geography: region, zone origin and destination (RDM, Super district, county) : Just the origin zone ?   
    
Non-work destinations accessible from people's homes
    count of Non-work-destinations accessible to each zone within isochrone thresholds (both actual and perceived). Weighted average by population in origin zone.
    PP metric: yes
    Multi or Single path : Single
    Modes : rail inclusive (Does this mean only rail OD pairs (IVTHWY + IVTCOM > 0) ?)
    Purpose split: No
    Period splits: Yes, average weekday and annual
    Geography: region, zone origin and destination (RDM, Super district, county) : Just the origin zone ?       

In [25]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)
    
_join = os.path.join
_dir = os.path.dirname
_norm = os.path.normpath

# paths
model_outputs_dir = params['model_dir']

skims_dir = _join(model_outputs_dir, "skims")
#landuse_dir = _join(model_outputs_dir, params['zone_file'])

#hwyskmMD = _join(params['best_path_skim_dir'], 'am_KNR_TRN_WLK_v9_1_release11302022_bestpathresults.omx')

#transit skims - names
#acc_egr = params['access_egress_modes']
#transit_skim_files = []
#for per in params['periods']:
#    for acc in acc_egr:
#        file_name = _join(params['best_path_skim_dir'], per+'_'+acc+'_v9_1_release11302022_bestpathresults.omx')
#        transit_skim_files.append(file_name)

#transit_skim_files = [r'C:\VY-Projects\Link21\BaseYear2015\Best Single Path\am_KNR_TRN_WLK_v9_1_release11302022_bestpathresults.omx']
#cores - 'BOARDS', 'DDIST', 'DTIME', 'FARE', 'IVT', 'IVTCOM', 'IVTEXP', 'IVTFRY', 'IVTHVY', 'IVTLOC', 'IVTLRT',
# 'IWAIT', 'PIVTCOM', 'PIVTEXP', 'PIVTFRY', 'PIVTHVY', 'PIVTLOC', 'PIVTLRT', 'WACC', 'WAIT', 'WAUX', 'WEGR', 'XWAIT'

ctramp_dir = params['ctramp_dir']
actual_tt_cores = params['total_travel_time']
perceived_tt_cores = params['perceived_travel_time']
time_periods = params['periods']
acc_egr = params['access_egress_modes']

concept_id = params['concept_id']
summary_columns = params['final_columns']

preprocess_dir = _join(ctramp_dir, '_pre_process_files')
perf_measure_columns = params['final_columns']
period = params['periods']
summary_dir = params['summary_dir']

best_path_skim_extension = params['best_path_skim_extension']

annual_transit_factor = params['annual_transit_factor']
annual_auto_factor = params['annual_auto_factor']

filename_extension = params['filename_extension']

In [ ]:
#skm = omx.open_file(transit_skim_files[4])
#print(transit_skim_files[4])
#df = skm.list_matrices()

In [ ]:
#skm.shape()

In [12]:
#Read data

#taz to RDM zones, super districts, county
geo_cwks = pd.read_csv(_join(params['common_dir'], "geographies.csv")) #columns taz, rdm_zones, super_district, county

#taz to priority population
pp_perc = pd.read_excel(_join(params['common_dir'], "TAZ_Tract_cwk_summary.xlsx")) #columns = taz, pp_share 

#get geogrpahies and priority population in the same file
geo_pp_cwks = pd.merge(geo_cwks, pp_perc, on = 'taz', how = 'left') 

#transbay od pairs
transbay_od = pd.read_csv(_join(params['common_dir'], "transbay_od.csv")) #columns = transbay_o, transbay_d

#non work destinations tazs
non_work_tazs = pd.read_excel(_join(params['common_dir'], 'non_work_destinations.xlsx'))
#non_work_tazs = list(non_work_tazs['non_wrk_taz'])

#taz data for empoyments and resindent employments
tazData = pd.read_csv(_join(params['model_dir'], params['zone_file']))
tazDataTotemp = tazData[["ZONE", "TOTEMP", "RETEMPN", "FPSEMPN", "HEREMPN", "OTHEMPN", "AGREMPN", "MWTEMPN"]]
tazDataEmpres = tazData[["ZONE", "EMPRES"]]
tazDataPop = tazData[["ZONE", "TOTPOP"]]

#mat_core = params['connectivity_mat_core']
time_thresholds = params['accessibility_thresholds']

In [4]:
#ck = omx.open_file(transit_skim_files[0])
#ck.list_matrices()

all_tod_tt = pd.read_parquet(_join(preprocess_dir, 'tod_min_actual_travel_time.parquet'))
perc_tod_tt = pd.read_parquet(_join(preprocess_dir, 'tod_min_perceived_travel_time.parquet'))

In [5]:
employment_categories = {'TOTEMP': 'Total employment',
                  'RETEMPN': 'Retail employment', 
                  'FPSEMPN': 'Financial and professional services employment', 
                  'HEREMPN': 'Health, educational and recreational service employment', 
                  'OTHEMPN': 'other employment',
                  'AGREMPN': 'agricultural and natural resources employment', 
                  'MWTEMPN': 'wholesale trade and transportation employment'}

In [11]:
for emp_cat in employment_categories:
    print(emp_cat, employment_categories[emp_cat])

TOTEMP Total employment
RETEMPN Retail employment
FPSEMPN Financial and professional services employment
HEREMPN Health, educational and recreational service employment
OTHEMPN other employment
AGREMPN agricultural and natural resources employment
MWTEMPN wholesale trade and transportation employment


In [ ]:
#retail trade employment; 
#financial and professional services employment;
#health, educational and recreational service employment;  
#agricultural and natural resources employment; 
#manufacturing, wholesale trade and transportation employment; and other employment.

In [19]:
all_tod_tt.columns

Index(['am', 'ea', 'ev', 'md', 'pm'], dtype='object', name='tp')

In [22]:
def get_accessibility_jobs(all_tod_tt, time_periods, time_thresholds, geo_pp_cwks, tt_verbose):

    acc_jobs = []
    acc_jobs_nwd = []
    df = []
    acc_jobs_pp = []
    acc_jobs_rdm = []
    acc_jobs_sd = []
    acc_jobs_cnty = []
    
    for period in time_periods:
        
            
        for emp_cat in employment_categories:
            print(period, emp_cat)
            emp = employment_categories[emp_cat]

            timedaData = all_tod_tt[[period]].reset_index()
            #print(timedaData.columns)
            timedaData = timedaData.fillna(0)
            timedaData[period] = timedaData[period]/100

            #total employment based on destination
            od_data = pd.merge(timedaData, tazDataTotemp, left_on='dest', right_on='ZONE', how = 'left')
            #print(od_data.columns)

            #for time thresholds - currently set in config files
            for threshold in time_thresholds:
                # print(f'processing {transit_file} for time threshold {threshold}')
                # create data for all destinations
                oddata_min = od_data.loc[od_data[period] <= threshold]
                oddata_min = oddata_min.groupby(['orig'])[emp_cat].sum().reset_index()

                # adding employed residents
                oddata_min = pd.merge(oddata_min, tazDataEmpres, left_on= ['orig'], right_on =['ZONE'], how ='left')

                # regional 
                oddata_min['wt_empres'] = oddata_min[emp_cat] * oddata_min['EMPRES']
                job_acc_min = oddata_min['wt_empres'].sum()/oddata_min['EMPRES'].sum()

                #acc_jobs.append(job_acc_min)
                df_temp_region = pd.DataFrame({'Description': f"{emp} accessible within " + tt_verbose + " " + str(threshold) + " minutes",
                                               'Population': 'Whole Population',
                                               'Period': period,
                                               'Geography': 'Regional total',
                                               'Zone_ID': 'Region',
                                               'Submetric': 'B1.1.1',
                                               'Total_Increment': '',
                                               'Value': job_acc_min}, index=[0])
                df.append(df_temp_region)

                # adding priority population and geographies
                oddata_min = pd.merge(oddata_min, geo_pp_cwks, left_on= ['orig'], right_on =['taz'], how ='left')

                 # for county
                if 'county' in geo_pp_cwks.columns:
                    oddata_min_cnty = oddata_min.groupby(['county'])['wt_empres','EMPRES'].sum().reset_index()
                    oddata_min_cnty['Value'] =  oddata_min_cnty['wt_empres']/oddata_min_cnty['EMPRES']
                    oddata_min_cnty['Description'] = f"{emp} accessible within " + tt_verbose + " " + str(threshold) + " minutes"
                    oddata_min_cnty['Period'] = period
                    oddata_min_cnty['Geography'] = 'County'
                    oddata_min_cnty['Population'] = 'Whole Population'
                    oddata_min_cnty.rename(columns={'county': 'Zone_ID'}, inplace=True)
                    oddata_min_cnty = oddata_min_cnty[['Description', 'Population', 'Period',
                                                     'Geography', 'Zone_ID', 'Value']]   
                    oddata_min_cnty['Submetric'] = 'B1.1.2'
                    oddata_min_cnty['Total_Increment'] = ''
                    acc_jobs_cnty.append(oddata_min_cnty)

                # for RDM zones
                if 'rdm_zones' in geo_pp_cwks.columns:
                    oddata_min_rdm = oddata_min.groupby(['rdm_zones'])['wt_empres','EMPRES'].sum().reset_index()
                    oddata_min_rdm['Value'] =  oddata_min_rdm['wt_empres']/oddata_min_rdm['EMPRES']
                    oddata_min_rdm['Description'] = f"{emp} accessible within " + tt_verbose + " " + str(threshold) + " minutes"
                    oddata_min_rdm['Period'] = period
                    oddata_min_rdm['Geography'] = 'RDM'
                    oddata_min_rdm['Population'] = 'Whole Population'
                    oddata_min_rdm.rename(columns={'rdm_zones' : 'Zone_ID'}, inplace=True)
                    oddata_min_rdm = oddata_min_rdm[['Description', 'Population', 'Period',
                                                     'Geography', 'Zone_ID', 'Value']]
                    oddata_min_rdm['Submetric'] = 'B1.1.3'
                    oddata_min_rdm['Total_Increment'] = ''
                    acc_jobs_rdm.append(oddata_min_rdm)

                # for superdistrict
                if 'super_district' in geo_pp_cwks.columns:
                    oddata_min_sd = oddata_min.groupby(['super_district'])['wt_empres','EMPRES'].sum().reset_index()
                    oddata_min_sd['Value'] =  oddata_min_sd['wt_empres']/oddata_min_sd['EMPRES']
                    oddata_min_sd['Description'] = f"{emp} accessible within " + tt_verbose + " " + str(threshold) + " minutes"
                    oddata_min_sd['Period'] = period
                    oddata_min_sd['Population'] = 'Whole Population'
                    oddata_min_sd['Geography'] = 'Superdistrict'
                    oddata_min_sd.rename(columns={'super_district': 'Zone_ID'}, inplace=True)
                    oddata_min_sd = oddata_min_sd[['Description', 'Population', 'Period',
                                                     'Geography', 'Zone_ID', 'Value']]
                    oddata_min_sd['Submetric'] = 'B1.1.4'
                    oddata_min_sd['Total_Increment'] = ''
                    acc_jobs_sd.append(oddata_min_sd)

                # for prioirty population
                if 'pp_share' in geo_pp_cwks.columns:
                    oddata_min['priority_population'] = oddata_min['pp_share'].apply(lambda x: 1 if x > 0 else 0)
                    oddata_min['wt_empres_pp'] = oddata_min[emp_cat] * oddata_min['EMPRES'] * oddata_min['pp_share']/100
                    oddata_min['EMPRES_pp'] = oddata_min['EMPRES'] * oddata_min['pp_share']/100
                    oddata_min_pp = oddata_min[oddata_min['priority_population']==1]
                    oddata_min_nonpp = oddata_min[oddata_min['priority_population']==0]
                    oddata_min_pp = oddata_min_pp.groupby(['priority_population'])['wt_empres_pp','EMPRES_pp'].sum().reset_index()
                    oddata_min_pp['Value'] = oddata_min_pp['wt_empres_pp']/oddata_min_pp['EMPRES_pp']
                    oddata_min_pp = oddata_min_pp[['priority_population', 'Value']]

                    oddata_min_nonpp = oddata_min_nonpp.groupby(['priority_population'])['wt_empres','EMPRES'].sum().reset_index()
                    oddata_min_nonpp['Value'] = oddata_min_nonpp['wt_empres']/oddata_min_nonpp['EMPRES']
                    oddata_min_nonpp = oddata_min_nonpp[['priority_population', 'Value']]

                    oddata_min_comb = pd.concat([oddata_min_pp, oddata_min_nonpp], ignore_index=False)
                    oddata_min_comb = oddata_min_comb.loc[oddata_min_comb['priority_population'] == 1]
                    oddata_min_comb = oddata_min_comb.drop(columns=['priority_population'])
                    oddata_min_comb['Description'] = f"{emp} accessible within " + tt_verbose + " " + str(threshold) + " minutes"
                    oddata_min_comb['Period'] = period
                    oddata_min_comb['Population'] = 'Priority population'
                    oddata_min_comb['Geography'] = 'Regional total'
                    oddata_min_comb['Zone_ID'] = 'Region'
                    oddata_min_comb = oddata_min_comb[['Description', 'Population', 'Period', 
                                                     'Geography', 'Zone_ID', 'Value']]
                    oddata_min_comb['Submetric'] = 'B1.1.5'
                    oddata_min_comb['Total_Increment'] = ''
                    acc_jobs_pp.append(oddata_min_comb)
            
    df_region = pd.concat(df).reset_index(drop=True)
    df_pp = pd.concat(acc_jobs_pp).reset_index(drop=True)
    df_rdm = pd.concat(acc_jobs_rdm).reset_index(drop=True)
    df_sd = pd.concat(acc_jobs_sd).reset_index(drop=True)
    df_cnty = pd.concat(acc_jobs_cnty).reset_index(drop=True)
    
    return df_region, df_pp, df_rdm, df_sd, df_cnty

In [43]:
df_region, df_pp, df_rdm, df_sd, df_cnty = get_accessibility_jobs(all_tod_tt, time_periods, time_thresholds, 
                                                                  geo_pp_cwks, "actual")

am TOTEMP
am RETEMPN
am FPSEMPN
am HEREMPN
am OTHEMPN
am AGREMPN
am MWTEMPN
md TOTEMP
md RETEMPN
md FPSEMPN
md HEREMPN
md OTHEMPN
md AGREMPN
md MWTEMPN
pm TOTEMP
pm RETEMPN
pm FPSEMPN
pm HEREMPN
pm OTHEMPN
pm AGREMPN
pm MWTEMPN
ev TOTEMP
ev RETEMPN
ev FPSEMPN
ev HEREMPN
ev OTHEMPN
ev AGREMPN
ev MWTEMPN
ea TOTEMP
ea RETEMPN
ea FPSEMPN
ea HEREMPN
ea OTHEMPN
ea AGREMPN
ea MWTEMPN


In [44]:
decimals = 0  
#df['column'] = df['column'].apply(lambda x: round(x, decimals))

In [45]:
comb_df = pd.concat([df_region, df_pp, df_rdm, df_sd, df_cnty], ignore_index=True)
comb_df['Concept_ID'] = concept_id
comb_df['Metric_ID'] = 'B1.1'
comb_df['Metric_name'] = 'Jobs accessible from peoples homes - actual travel time'
comb_df['Origin_zone'] = ''
comb_df['Dest_zone'] = ''
comb_df['Purpose'] = ''
comb_df['Units'] = 'Jobs'
comb_df['Value'] = comb_df['Value'].apply(lambda x: round(x, decimals))
comb_df_act = comb_df[summary_columns]

In [46]:
unique_medtric_ids = comb_df_act['Submetric'].unique()
for mids in unique_medtric_ids:
    metric_name = 'jobs_accessible_from_home_actual_time_'
    dfs = comb_df_act.loc[comb_df_act['Submetric']==mids]
    #print(dfs.columns)
    dfs = dfs[perf_measure_columns]
    file_name = mids
    dfs.to_csv(_join(summary_dir, metric_name + file_name + filename_extension +  '.csv'), index=None)
    print(len(dfs), file_name, dfs['Value'].sum())

comb_df_act.to_csv(_join(summary_dir, 'jobs_accessible_from_home_actual_time_' + 'B1.1' + filename_extension + '.csv'), index=None)

140 B1.1.1 37323070.0
140 B1.1.5 38806332.0
34720 B1.1.3 10240529960.0
4760 B1.1.4 1400442320.0
1260 B1.1.2 401669633.0


In [47]:
df_region, df_pp, df_rdm, df_sd, df_cnty = get_accessibility_jobs(perc_tod_tt, time_periods, time_thresholds, 
                                                                  geo_pp_cwks, "perceived")

am TOTEMP
am RETEMPN
am FPSEMPN
am HEREMPN
am OTHEMPN
am AGREMPN
am MWTEMPN
md TOTEMP
md RETEMPN
md FPSEMPN
md HEREMPN
md OTHEMPN
md AGREMPN
md MWTEMPN
pm TOTEMP
pm RETEMPN
pm FPSEMPN
pm HEREMPN
pm OTHEMPN
pm AGREMPN
pm MWTEMPN
ev TOTEMP
ev RETEMPN
ev FPSEMPN
ev HEREMPN
ev OTHEMPN
ev AGREMPN
ev MWTEMPN
ea TOTEMP
ea RETEMPN
ea FPSEMPN
ea HEREMPN
ea OTHEMPN
ea AGREMPN
ea MWTEMPN


In [48]:
comb_df = pd.concat([df_region, df_pp, df_rdm, df_sd, df_cnty], ignore_index=True)

comb_df['Concept_ID'] = concept_id
comb_df['Metric_ID'] = 'B1.1'
comb_df['Metric_name'] = 'Jobs accessible from peoples homes - perceived travel time'
comb_df['Origin_zone'] = ''
comb_df['Dest_zone'] = ''
comb_df['Purpose'] = ''
comb_df['Units'] = 'Jobs'
comb_df['Value'] = comb_df['Value'].apply(lambda x: round(x, decimals))

comb_df_perc = comb_df[summary_columns]

In [49]:
unique_medtric_ids = comb_df_perc['Submetric'].unique()
for mids in unique_medtric_ids:
    metric_name = 'jobs_accessible_from_home_perceived_time_'
    dfs = comb_df_act.loc[comb_df_act['Submetric']==mids]
    #print(dfs.columns)
    dfs = dfs[perf_measure_columns]
    file_name = mids
    dfs.to_csv(_join(summary_dir, metric_name + file_name + filename_extension +  '.csv'), index=None)
    print(len(dfs), file_name, dfs['Value'].sum())

comb_df_act.to_csv(_join(summary_dir, 'jobs_accessible_from_home_perceived_time_' + 'B1.1' + filename_extension + '.csv'), index=None)

140 B1.1.1 37323070.0
140 B1.1.5 38806332.0
34720 B1.1.3 10240529960.0
4760 B1.1.4 1400442320.0
1260 B1.1.2 401669633.0


def get_non_work_destionations(all_tod_tt, time_periods, time_thresholds, geo_pp_cwks, non_work_dest_tazs, tt_verbose):

    acc_jobs = []
    df = []
    nwd_acc_min = []
    nwd_jobs_pp = []
    nwd_jobs_rdm = []
    nwd_jobs_sd = []
    nwd_jobs_cnty = []
    
    for period in time_periods:
        
        if period in all_tod_tt.columns:

            timedaData = all_tod_tt[[period]].reset_index()
            print(timedaData.columns)
            timedaData = timedaData.fillna(0)
            timedaData = timedaData.loc[timedaData[period]>0]
            timedaData[period] = timedaData[period]/100

            #total non work destinations based on destination
            od_data = pd.merge(timedaData, non_work_dest_tazs, left_on='dest', right_on='taz', how = 'inner')
            od_data.isna().sum()

            #for time thresholds - currently set in config files
            for threshold in time_thresholds:
                # print(f'processing {transit_file} for time threshold {threshold}')
                # create data for all destinations
                oddata_min = od_data.loc[od_data[period] <= threshold]
                oddata_min = oddata_min.groupby(['orig'])['non_work_dest'].sum().reset_index()

                # adding employed residents
                oddata_min = pd.merge(oddata_min, tazDataPop, left_on= ['orig'], right_on =['ZONE'], how ='left')

                # regional 
                oddata_min['wt_pop'] = oddata_min['non_work_dest'] * oddata_min['TOTPOP']
                nwd_acc_min = oddata_min['wt_pop'].sum()/oddata_min['non_work_dest'].sum()
                #acc_jobs.append(job_acc_min)

                df_temp_region = pd.DataFrame({'Description': "Non-work destinations accessible within " + tt_verbose + " " + str(threshold) + " minutes",
                                               'Population': 'Whole Population',
                                               'Period': period,
                                               'Geography': 'Regional total',
                                               'Zone_ID': 'Region',
                                               'Submetric': 'B1.3.1',
                                               'Total_Increment': '',
                                               'Value': nwd_acc_min}, index=[0])
                
                
                df.append(df_temp_region)

                # adding priority population and geographies
                oddata_min = pd.merge(oddata_min, geo_pp_cwks, left_on= ['orig'], right_on =['taz'], how ='left')

                # for prioirty population
                if 'pp_share' in geo_pp_cwks.columns:
                    oddata_min['priority_population'] = oddata_min['pp_share'].apply(lambda x: 1 if x > 0 else 0)
                    oddata_min['wt_pop_pp'] = oddata_min['non_work_dest'] * oddata_min['TOTPOP'] * oddata_min['pp_share']/100
                    oddata_min['pop_pp'] = oddata_min['TOTPOP'] * oddata_min['pp_share']/100
                    oddata_min_pp = oddata_min[oddata_min['priority_population']==1]
                    oddata_min_nonpp = oddata_min[oddata_min['priority_population']==0]
                    oddata_min_pp = oddata_min_pp.groupby(['priority_population'])['wt_pop_pp','pop_pp'].sum().reset_index()
                    oddata_min_pp['Value'] = oddata_min_pp['wt_pop_pp']/oddata_min_pp['pop_pp']
                    oddata_min_pp = oddata_min_pp[['priority_population', 'Value']]

                    oddata_min_nonpp = oddata_min_nonpp.groupby(['priority_population'])['wt_pop','TOTPOP'].sum().reset_index()
                    oddata_min_nonpp['Value'] = oddata_min_nonpp['wt_pop']/oddata_min_nonpp['TOTPOP']
                    oddata_min_nonpp = oddata_min_nonpp[['priority_population', 'Value']]

                    oddata_min_comb = pd.concat([oddata_min_pp, oddata_min_nonpp], ignore_index=False)
                    oddata_min_comb = oddata_min_comb.loc[oddata_min_comb['priority_population'] == 1]
                    oddata_min_comb = oddata_min_comb.drop(columns=['priority_population'])
                    oddata_min_comb['Description'] = "Non-work destinations accessible within " + tt_verbose + " " + str(threshold) + " minutes"
                    oddata_min_comb['Period'] = period
                    oddata_min_comb['Population'] = 'Priority population'
                    oddata_min_comb['Geography'] = 'Regional total'
                    oddata_min_comb['Zone_ID'] = 'Region'
                    oddata_min_comb = oddata_min_comb[['Description', 'Population', 'Period', 
                                                     'Geography', 'Zone_ID', 'Value']]
                    oddata_min_comb['Submetric'] = 'B1.3.2'
                    oddata_min_comb['Total_Increment'] = ''
                    nwd_jobs_pp.append(oddata_min_comb)

                # for RDM zones
                if 'rdm_zones' in geo_pp_cwks.columns:
                    oddata_min_rdm = oddata_min.groupby(['rdm_zones'])['wt_pop','TOTPOP'].sum().reset_index()
                    oddata_min_rdm['Value'] =  oddata_min_rdm['wt_pop']/oddata_min_rdm['TOTPOP']
                    oddata_min_rdm['Description'] = "Non-work destinations accessible within " + tt_verbose + " " + str(threshold) + " minutes"
                    oddata_min_rdm['Period'] = period
                    oddata_min_rdm['Geography'] = 'RDM'
                    oddata_min_rdm['Population'] = 'Whole Population'
                    oddata_min_rdm.rename(columns={'rdm_zones' : 'Zone_ID'}, inplace=True)
                    oddata_min_rdm = oddata_min_rdm[['Description', 'Population', 'Period',
                                                     'Geography', 'Zone_ID', 'Value']]
                    oddata_min_rdm['Submetric'] = 'B1.3.3'
                    oddata_min_rdm['Total_Increment'] = ''
                    nwd_jobs_rdm.append(oddata_min_rdm)

                # for superdistrict
                if 'super_district' in geo_pp_cwks.columns:
                    oddata_min_sd = oddata_min.groupby(['super_district'])['wt_pop','TOTPOP'].sum().reset_index()
                    oddata_min_sd['Value'] =  oddata_min_sd['wt_pop']/oddata_min_sd['TOTPOP']
                    oddata_min_sd['Description'] = "Non-work destinations accessible within " + tt_verbose + " " + str(threshold) + " minutes"
                    oddata_min_sd['Period'] = period
                    oddata_min_sd['Population'] = 'Whole Population'
                    oddata_min_sd['Geography'] = 'Superdistrict'
                    oddata_min_sd.rename(columns={'super_district': 'Zone_ID'}, inplace=True)
                    oddata_min_sd = oddata_min_sd[['Description', 'Population', 'Period',
                                                     'Geography', 'Zone_ID', 'Value']]
                    oddata_min_sd['Submetric'] = 'B1.3.4'
                    oddata_min_sd['Total_Increment'] = ''
                    nwd_jobs_sd.append(oddata_min_sd)

                # for county
                if 'county' in geo_pp_cwks.columns:
                    oddata_min_cnty = oddata_min.groupby(['county'])['wt_pop','TOTPOP'].sum().reset_index()
                    oddata_min_cnty['Value'] =  oddata_min_cnty['wt_pop']/oddata_min_cnty['TOTPOP']
                    oddata_min_cnty['Description'] = "Non-work destinations accessible within " + tt_verbose + " " + str(threshold) + " minutes"
                    oddata_min_cnty['Period'] = period
                    oddata_min_cnty['Geography'] = 'County'
                    oddata_min_cnty['Population'] = 'Whole Population'
                    oddata_min_cnty.rename(columns={'county': 'Zone_ID'}, inplace=True)
                    oddata_min_cnty = oddata_min_cnty[['Description', 'Population', 'Period',
                                                     'Geography', 'Zone_ID', 'Value']]   
                    oddata_min_cnty['Submetric'] = 'B1.3.5'
                    oddata_min_cnty['Total_Increment'] = ''
                    nwd_jobs_cnty.append(oddata_min_cnty)
                        
        else:
            print(f'{period} travel time doesn\'t exist.')
            
    df_region = pd.concat(df).reset_index(drop=True)
    df_pp = pd.concat(nwd_jobs_pp).reset_index(drop=True)
    df_rdm = pd.concat(nwd_jobs_rdm).reset_index(drop=True)
    df_sd = pd.concat(nwd_jobs_sd).reset_index(drop=True)
    df_cnty = pd.concat(nwd_jobs_cnty).reset_index(drop=True)
    
    return df_region, df_pp, df_rdm, df_sd, df_cnty

df_region, df_pp, df_rdm, df_sd, df_cnty = get_non_work_destionations(all_tod_tt, time_periods, time_thresholds,
                                                                      geo_pp_cwks, non_work_tazs, "actual")

comb_df = pd.concat([df_region, df_pp, df_rdm, df_sd, df_cnty], ignore_index=True)

comb_df['Concept_ID'] = concept_id
comb_df['Metric_ID'] = 'B1.3'
comb_df['Metric_name'] = 'Non-work destinations accessible from people\'s homes'
comb_df['Origin_zone'] = ''
comb_df['Dest_zone'] = ''
comb_df['Purpose'] = ''
comb_df['Units'] = 'Destinations'
comb_df['Value'] = comb_df['Value'].apply(lambda x: round(x, decimals))

comb_df_nwd_act = comb_df[summary_columns]

df_region, df_pp, df_rdm, df_sd, df_cnty = get_non_work_destionations(perc_tod_tt, time_periods, time_thresholds,
                                                                      geo_pp_cwks, non_work_tazs, "perceived")

comb_df = pd.concat([df_region, df_pp, df_rdm, df_sd, df_cnty], ignore_index=True)

comb_df['Concept_ID'] = concept_id
comb_df['Metric_ID'] = 'B1.3'
comb_df['Metric_name'] = 'Non-work destinations accessible from people\'s homes'
comb_df['Origin_zone'] = ''
comb_df['Dest_zone'] = ''
comb_df['Purpose'] = ''
comb_df['Units'] = 'Destinations'
comb_df['Value'] = comb_df['Value'].apply(lambda x: round(x, decimals))

comb_df_nwd_perc = comb_df[summary_columns]

comb_df_nwd = pd.concat([comb_df_nwd_act, comb_df_nwd_perc], ignore_index=True)

In [ ]:
#comb_df = pd.concat([comb_df_acc, comb_df_nwd], ignore_index=True)
#comb_df = comb_df[summary_columns]

In [ ]:
#with pd.ExcelWriter(os.path.join(summary_outputs, 'concept-BY15.xlsx'), engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
#    comb_df_acc.to_excel(writer, sheet_name='B1.1', startcol=0, index=False)
#    comb_df_nwd.to_excel(writer, sheet_name='B1.3', startcol=0, index=False)

## Archive

def get_accessibility_jobs(transit_skim_files, mat_core, geo_pp_cwks, non_work_dest_tazs):

    acc_jobs = []
    acc_jobs_nwd = []
    df = []
    acc_jobs_pp = []
    acc_jobs_rdm = []
    acc_jobs_sd = []
    acc_jobs_cnty = []
    
    #for 25 transit skims files - this includes time periods
    for transit_file in transit_skim_files:
        
        transit_file_name = os.path.split(transit_file)[-1]
        
        for core in mat_core:
            # od data from skims
            timedaData = convertMat2Df(transit_file, core)
            timedaData = timedaData.loc[timedaData[core]>0]
            timedaData[core] = timedaData[core]/100
              
            #total employment based on destination
            od_data = pd.merge(timedaData, tazDataTotemp, left_on='dest', right_on='ZONE', how = 'left')
            od_data.columns

            #for time thresholds - currently set in config files
            for threshold in time_thresholds:
                print(f'processing {transit_file} for time threshold {threshold}')
                # create data for all destinations
                oddata_min = od_data.loc[od_data[core] <= threshold]
                oddata_min = oddata_min.groupby(['orig'])['TOTEMP'].sum().reset_index()

                # adding employed residents
                oddata_min = pd.merge(oddata_min, tazDataEmpres, left_on= ['orig'], right_on =['ZONE'], how ='left')

                # regional 
                oddata_min['wt_empres'] = oddata_min['TOTEMP'] * oddata_min['EMPRES']
                job_acc_min = oddata_min['wt_empres'].sum()/oddata_min['EMPRES'].sum()
                #acc_jobs.append(job_acc_min)
                
                df_temp_region = pd.DataFrame({'transit_file': os.path.split(transit_file)[-1], 'time_threshold': threshold, 'jobs_from_home': job_acc_min, 'time': core}, index=[0])
                df.append(df_temp_region)

                # adding priority population and geographies
                oddata_min = pd.merge(oddata_min, geo_pp_cwks, left_on= ['orig'], right_on =['taz'], how ='left')
                
                # for prioirty population
                if 'pp_share' in geo_pp_cwks.columns:
                    oddata_min['priority_population'] = oddata_min['pp_share'].apply(lambda x: 1 if x > 0 else 0)
                    oddata_min['wt_empres_pp'] = oddata_min['TOTEMP'] * oddata_min['EMPRES'] * oddata_min['pp_share']/100
                    oddata_min['EMPRES_pp'] = oddata_min['EMPRES'] * oddata_min['pp_share']/100
                    oddata_min_pp = oddata_min[oddata_min['priority_population']==1]
                    oddata_min_nonpp = oddata_min[oddata_min['priority_population']==0]
                    oddata_min_pp = oddata_min_pp.groupby(['priority_population'])['wt_empres_pp','EMPRES_pp'].sum().reset_index()
                    oddata_min_pp['jobs_from_home'] = oddata_min_pp['wt_empres_pp']/oddata_min_pp['EMPRES_pp']
                    oddata_min_pp = oddata_min_pp[['priority_population', 'jobs_from_home']]

                    oddata_min_nonpp = oddata_min_nonpp.groupby(['priority_population'])['wt_empres','EMPRES'].sum().reset_index()
                    oddata_min_nonpp['jobs_from_home'] = oddata_min_nonpp['wt_empres']/oddata_min_nonpp['EMPRES']
                    oddata_min_nonpp = oddata_min_nonpp[['priority_population', 'jobs_from_home']]

                    oddata_min_comb = pd.concat([oddata_min_pp, oddata_min_nonpp], ignore_index=False)
                    oddata_min_comb['time_threshold'] = threshold
                    oddata_min_comb['transit_file'] = transit_file_name
                    oddata_min_comb['time'] = core
                    acc_jobs_pp.append(oddata_min_comb)

                # for RDM zones
                if 'rdm_zones' in geo_pp_cwks.columns:
                    oddata_min_rdm = oddata_min.groupby(['rdm_zones'])['wt_empres','EMPRES'].sum().reset_index()
                    oddata_min_rdm['jobs_from_home'] =  oddata_min_rdm['wt_empres']/oddata_min_rdm['EMPRES']
                    oddata_min_rdm['time_threshold'] = threshold
                    oddata_min_rdm['transit_file'] = transit_file_name
                    oddata_min_rdm['time'] = core
                    oddata_min_rdm = oddata_min_rdm[['rdm_zones', 'jobs_from_home', 'time_threshold', 'transit_file', 'time']] 
                    acc_jobs_rdm.append(oddata_min_rdm)

                # for superdistrict
                if 'super_district' in geo_pp_cwks.columns:
                    oddata_min_sd = oddata_min.groupby(['super_district'])['wt_empres','EMPRES'].sum().reset_index()
                    oddata_min_sd['jobs_from_home'] =  oddata_min_sd['wt_empres']/oddata_min_sd['EMPRES']
                    oddata_min_sd['time_threshold'] = threshold
                    oddata_min_sd['transit_file'] = transit_file_name
                    oddata_min_sd['time'] = core
                    oddata_min_sd = oddata_min_sd[['super_district', 'jobs_from_home', 'time_threshold', 'transit_file', 'time']] 
                    acc_jobs_sd.append(oddata_min_sd)

                # for county
                if 'county' in geo_pp_cwks.columns:
                    oddata_min_cnty = oddata_min.groupby(['county'])['wt_empres','EMPRES'].sum().reset_index()
                    oddata_min_cnty['jobs_from_home'] =  oddata_min_cnty['wt_empres']/oddata_min_cnty['EMPRES']
                    oddata_min_cnty['time_threshold'] = threshold
                    oddata_min_cnty['transit_file'] = transit_file_name
                    oddata_min_cnty['time'] = core
                    oddata_min_cnty = oddata_min_cnty[['county', 'jobs_from_home', 'time_threshold', 'transit_file', 'time']] 
                    acc_jobs_cnty.append(oddata_min_cnty)
            
    df_region = pd.concat(df)
    df_pp = pd.concat(acc_jobs_pp)
    df_rdm = pd.concat(acc_jobs_rdm)
    df_sd = pd.concat(acc_jobs_sd)
    df_cnty = pd.concat(acc_jobs_cnty)
    
    return df_region, df_pp, df_rdm, df_sd, df_cnty

def get_non_work_destionations(transit_skim_files, mat_core, geo_pp_cwks, non_work_dest_tazs):

    acc_jobs = []
    nwd_jobs_nwd = []
    df = []
    nwd_jobs_pp = []
    nwd_jobs_rdm = []
    nwd_jobs_sd = []
    nwd_jobs_cnty = []
    
    
    #for 25 transit skims files - this includes time periods
    for transit_file in transit_skim_files:
        
        transit_file_name = os.path.split(transit_file)[-1]
        
        for core in mat_core:
            # od data from skims
            timedaData = convertMat2Df(transit_file, core)
            timedaData = timedaData.loc[timedaData[core]>0]
            timedaData[core] = timedaData[core]/100

            #total non work destinations based on destination
            od_data = pd.merge(timedaData, non_work_dest_tazs, left_on='dest', right_on='taz', how = 'inner')
            od_data.isna().sum()

            #for time thresholds - currently set in config files
            for threshold in time_thresholds:
                print(f'processing {transit_file} for time threshold {threshold}')
                # create data for all destinations
                oddata_min = od_data.loc[od_data[core] <= threshold]
                oddata_min = oddata_min.groupby(['orig'])['non_work_dest'].sum().reset_index()

                # adding employed residents
                oddata_min = pd.merge(oddata_min, tazDataPop, left_on= ['orig'], right_on =['ZONE'], how ='left')

                # regional 
                oddata_min['wt_pop'] = oddata_min['non_work_dest'] * oddata_min['TOTPOP']
                nwd_acc_min = oddata_min['wt_pop'].sum()/oddata_min['non_work_dest'].sum()
                #acc_jobs.append(job_acc_min)
                
                df_temp_region = pd.DataFrame({'transit_file': os.path.split(transit_file)[-1], 'time_threshold': threshold, 'nwd_from_home': nwd_acc_min, 'time': core}, index=[0])
                df.append(df_temp_region)

                # adding priority population and geographies
                oddata_min = pd.merge(oddata_min, geo_pp_cwks, left_on= ['orig'], right_on =['taz'], how ='left')
                
                # for prioirty population
                if 'pp_share' in geo_pp_cwks.columns:
                    oddata_min['priority_population'] = oddata_min['pp_share'].apply(lambda x: 1 if x > 0 else 0)
                    oddata_min['wt_pop_pp'] = oddata_min['non_work_dest'] * oddata_min['TOTPOP'] * oddata_min['pp_share']/100
                    oddata_min['pop_pp'] = oddata_min['TOTPOP'] * oddata_min['pp_share']/100
                    oddata_min_pp = oddata_min[oddata_min['priority_population']==1]
                    oddata_min_nonpp = oddata_min[oddata_min['priority_population']==0]
                    oddata_min_pp = oddata_min_pp.groupby(['priority_population'])['wt_pop_pp','pop_pp'].sum().reset_index()
                    oddata_min_pp['nwd_from_home'] = oddata_min_pp['wt_pop_pp']/oddata_min_pp['pop_pp']
                    oddata_min_pp = oddata_min_pp[['priority_population', 'nwd_from_home']]

                    oddata_min_nonpp = oddata_min_nonpp.groupby(['priority_population'])['wt_pop','TOTPOP'].sum().reset_index()
                    oddata_min_nonpp['nwd_from_home'] = oddata_min_nonpp['wt_pop']/oddata_min_nonpp['TOTPOP']
                    oddata_min_nonpp = oddata_min_nonpp[['priority_population', 'nwd_from_home']]

                    oddata_min_comb = pd.concat([oddata_min_pp, oddata_min_nonpp], ignore_index=False)
                    oddata_min_comb['time_threshold'] = threshold
                    oddata_min_comb['transit_file'] = transit_file_name
                    oddata_min_comb['time'] = core
                    nwd_jobs_pp.append(oddata_min_comb)

                # for RDM zones
                if 'rdm_zones' in geo_pp_cwks.columns:
                    oddata_min_rdm = oddata_min.groupby(['rdm_zones'])['wt_pop','TOTPOP'].sum().reset_index()
                    oddata_min_rdm['nwd_from_home'] =  oddata_min_rdm['wt_pop']/oddata_min_rdm['TOTPOP']
                    oddata_min_rdm['time_threshold'] = threshold
                    oddata_min_rdm['transit_file'] = transit_file_name
                    oddata_min_rdm['time'] = core
                    oddata_min_rdm = oddata_min_rdm[['rdm_zones', 'nwd_from_home', 'time_threshold', 'transit_file', 'time']] 
                    nwd_jobs_rdm.append(oddata_min_rdm)

                # for superdistrict
                if 'super_district' in geo_pp_cwks.columns:
                    oddata_min_sd = oddata_min.groupby(['super_district'])['wt_pop','TOTPOP'].sum().reset_index()
                    oddata_min_sd['nwd_from_home'] =  oddata_min_sd['wt_pop']/oddata_min_sd['TOTPOP']
                    oddata_min_sd['time_threshold'] = threshold
                    oddata_min_sd['transit_file'] = transit_file_name
                    oddata_min_sd['time'] = core
                    oddata_min_sd = oddata_min_sd[['super_district', 'nwd_from_home', 'time_threshold', 'transit_file', 'time']] 
                    nwd_jobs_sd.append(oddata_min_sd)

                # for county
                if 'county' in geo_pp_cwks.columns:
                    oddata_min_cnty = oddata_min.groupby(['county'])['wt_pop','TOTPOP'].sum().reset_index()
                    oddata_min_cnty['nwd_from_home'] =  oddata_min_cnty['wt_pop']/oddata_min_cnty['TOTPOP']
                    oddata_min_cnty['time_threshold'] = threshold
                    oddata_min_cnty['transit_file'] = transit_file_name
                    oddata_min_cnty['time'] = core
                    oddata_min_cnty = oddata_min_cnty[['county', 'nwd_from_home', 'time_threshold', 'transit_file', 'time']] 
                    nwd_jobs_cnty.append(oddata_min_cnty)
            
    df_region = pd.concat(df)
    df_pp = pd.concat(nwd_jobs_pp)
    df_rdm = pd.concat(nwd_jobs_rdm)
    df_sd = pd.concat(nwd_jobs_sd)
    df_cnty = pd.concat(nwd_jobs_cnty)
    
    return df_region, df_pp, df_rdm, df_sd, df_cnty